In [2]:
# Распред обручей
import pandas as pd
import numpy as np
import random

shops = pd.read_excel('C:\анаконда\shop_list.xlsx', dtype='int')
                     
# достаем Тпг+размер c количеством которые нужно считать из накладной 
tmp_df = pd.read_excel('C:/рушники/123.xlsx', dtype='str')
tmp_df['tpg'] = tmp_df['Товарная подгруппа'].str.cat(tmp_df['Размер'], sep="_")

#del tmp_df['№ Поставщика']
tmp_df['tpg'] = tmp_df['tpg'].str.replace('.', ',')# меняем точки на запятые
tmp_df.sort_values(by=['tpg'], inplace=True)
tpg = tmp_df['tpg'].unique().tolist()
tpg.sort()
tpg_dic = tmp_df['tpg'].value_counts().to_dict()

# Загружаем нормы и остатки по нужным тпг чтобы посчитать дефицит
n_link =  r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx'

norms = pd.read_excel(n_link, sheet_name='гладкая_плоская размеры_тпг', skiprows=3, usecols=['номер', *tpg]) 
stock = pd.read_excel(r'C:\Остатки\Обручальные кольца.xlsx', usecols=['КодСклада','ТоварнаяПодгруппа','РазмерИзделия'], dtype='str')
stock.columns = ['КодСклада', 'Товарная подгруппа', 'Размер']
stock['Размер'] = stock['Размер'].str.replace('.', ',')
stock['tpg'] = stock['Товарная подгруппа'].str.cat(stock['Размер'], sep="_")
stock.drop(columns=['Товарная подгруппа','Размер'], inplace=True)
stock = stock[stock['tpg'].isin(tpg)]
stock = stock.groupby(by=['КодСклада', 'tpg'])['tpg'].aggregate('count').unstack(fill_value=0.0)
stock.reset_index(inplace=True)
stock = stock.astype('int')
stock.columns.values[0] = 'номер'
stock = stock[norms.columns.tolist()]
norms = shops.merge(norms, on='номер', how='left')
stock = shops.merge(stock, on='номер', how='left').fillna(0)
norms.set_index('номер', inplace=True)
stock.set_index('номер', inplace=True)

demand = norms - stock
demand = demand.applymap(lambda num: 0 if num < 0 else num)

def raspred_dict(df):
    emp = []
    cols = df.columns.tolist()
    for col in cols:
        emp.append(list(np.repeat(df.index.to_numpy(),df[col].values.astype(int))))
    return dict(zip(cols, emp))

no7171_dict = raspred_dict(demand)

def dictu(x):    
    if tpg_dic[x] > len(no7171_dict[x]):
        over = tpg_dic[x] - len(no7171_dict[x])
        no7171_dict[x] = [*no7171_dict[x], *([7171] * over)]        
    elif tpg_dic[x] < len(no7171_dict[x]):
        no7171_dict[x] = no7171_dict[x][:tpg_dic[x]]        
    return no7171_dict[x]

lst = [dictu(x) for x in tpg]

tmp_df['Период реализации'] = np.array([item for sublist in lst for item in sublist])
    


name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
tmp_df.to_excel(name, sheet_name='Движение товара', index=False)

In [4]:
# Распред букв или зодиаков по одному направлению
import pandas as pd
import numpy as np
import random

# get stockdata, goods receipt, shops list, stock capability
prih = pd.read_excel('C:/рушники/8786.xlsx')
prih.sort_values(by=['Товар'], inplace=True)
prih['Описание 3'] = prih['Описание 3'].str.lower()

letters = prih['Описание 3'].unique().tolist()
tovs = prih['Товар'].unique().tolist()
tovs.sort()

stock = pd.read_excel('C:/Остатки/Буквы и Зодиаки.xlsx')
stock = stock[stock['ТоварНо'].isin(tovs)]
stock.rename(columns={'Описание3': 'Описание 3'}, inplace=True)
stock['Описание 3'] = stock['Описание 3'].str.lower()

#create automatic demand calculation here
#norms = pd.read_excel('C:/Остатки/norm_bz.xlsx', sheet_name='norm_z', index_col= 'Название из NAV', usecols=['Название из NAV', 'sum_bk'])
#norm_dict = norms.sum_bk.apply(lambda x: round(x/12)).to_dict()

#estimate sheetname and filename for demand
tn = prih.loc[1, 'Товарное направление'].lower()
fname = 'C:/Остатки/' + prih.loc[1, 'Дизайн'][3:].lower() + '.xlsx'

demand = pd.read_excel(fname, sheet_name=tn, skiprows=1, usecols=['Наполняха', 'Код', *letters], index_col='Код')


#Create two dictionaries tov dict and letter dict
stock_dict = dict(zip(tovs,[stock[stock['ТоварНо'] == tov]['КодСклада'].tolist() for tov in tovs]))
df = prih.groupby(by=[ 'Товар', 'Описание 3'])['Товар'].count().rename('count')
df = df.reset_index()

tov_dict = dict(zip(df.Товар.tolist(), df[['Описание 3', 'count']].values.tolist())) #(tov:[letter, demand])

#letter: dataframe with columns:store number, filling rate, demand(positive num is overstock, negative num is demand)
letter_dict = dict(zip(letters,[demand[['Наполняха', letter]] for letter in letters]))

def get_shoplist(tov):
    """Get sorted by letter demand and stock list of stores with zero stock of tov.
    Changes letter dict according to this list. Two dictionaries needed to perform func"""
    
    letter = tov_dict[tov][0]
    lenth = tov_dict[tov][1]
    shoplist_full = letter_dict[letter].sort_values(by=[letter, 'Наполняха']).index.tolist()
    shoplist = [i for i in shoplist_full if i not in stock_dict[tov]]
    shop_len = len(shoplist)
    
    if shop_len > lenth:
        shoplist = shoplist[:lenth]
    elif len(shoplist) < lenth:
        over = lenth - shop_len
        shoplist = [*shoplist,*([7171] * over)]
    for shop in shoplist:
        if shop != 7171:
            letter_dict[letter][letter][shop] += 1
    return shoplist
      
lst = [get_shoplist(x) for x in tovs]
prih['Период реализации'] = np.array([item for sublist in lst for item in sublist])

name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
prih.to_excel(name, sheet_name='Движение товара', index=False)
    

C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
# Распред по товам по две три штуки в шоплист вторым столбцом указать количество столбец "кол"
import pandas as pd
import numpy as np
import random

# create distribution quantities dictionary
invoice = pd.read_excel('C:/рушники/9285.xlsx')
invoice.sort_values(by=['Товар'], inplace=True)
arts_dic = invoice['Товар'].value_counts().to_dict()

# create norms dataframe
shops = pd.read_excel('C:\анаконда\shop_list_doubled.xlsx', dtype='int')
shops.sort_values(by='кол',ascending=False, inplace=True)

num_dict = shops.кол.value_counts().sort_index(ascending=False).to_dict()
cols = list(arts_dic.keys())

norms = pd.concat(
[pd.DataFrame(
    data=np.full((num_dict[i], len(cols)), i), 
    columns=cols)
 for i in list(num_dict)]
)
norms.index = shops['номер']

# create stock df identical in shape to norms df
stock = pd.read_excel(r'C:\Остатки\Товы.xlsx', usecols=['КодСклада','ТоварНо'])
stock = stock[stock['КодСклада'].isin(norms.index.tolist()) & 
              stock['ТоварНо'].isin(cols)]

pivot = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()

stock = pd.DataFrame(data=shops['номер'].values, columns=['КодСклада']).merge(pivot, how='left')
stock.set_index('КодСклада',inplace=True)
stock.fillna(0, inplace=True)

if cols != stock.columns.to_list():
    add_cols = [x for x in cols if x not in stock.columns.to_list()]
    add_df = pd.DataFrame(data=np.zeros((stock.shape[0], len(add_cols))), columns=add_cols, index=stock.index)
    stock.join(add_df)

# demand 
demand = norms - stock

# distribution
def repeat_series(series):
    return np.repeat(series.index, series.values).tolist()

empty = []
for col in demand.columns.tolist():
    shop_lst = repeat_series(demand[col].loc[lambda x: x > 0])
    diff = arts_dic[col] - len(shop_lst)
    if diff > 0:
        shop_lst.extend(list(np.full(diff, 7171)))
    else:
        shop_lst = shop_lst[:arts_dic[col]]
    empty.extend(shop_lst)

invoice['Период реализации'] = empty

# save and exit
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
invoice.to_excel(name, sheet_name='Движение товара', index=False)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False